In [1]:
import pandas as pd
import numpy as np
import string
import re
import random
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer


In [2]:
df=pd.read_csv(r"D:\Capstone\Datasets\Data_preparation\preprocessed\Preprocessed_2000_samples.csv")


In [3]:
import ast

df['text_lemmatized'] = df['text_lemmatized'].apply(ast.literal_eval)

# Now you can proceed with concatenating the lists into sentences
df['sentences'] = df['text_lemmatized'].apply(lambda x: ' '.join(x))



In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorizer
vect = CountVectorizer()  
vects = vect.fit_transform(df.sentences)
vects.shape

(2000, 20782)

In [5]:
tdm = vects.T

term_document_matrix = pd.DataFrame.sparse.from_spmatrix(tdm, index=vect.get_feature_names_out(), columns=df['docid'].astype(str))

print(term_document_matrix.shape)

(20782, 2000)


## Max Normalization

In [6]:
import numpy as np

max_values = term_document_matrix.max(axis=0)
normalized_term_document_matrix =np.divide(term_document_matrix, max_values)
normalized_term_document_matrix=normalized_term_document_matrix.values


## TruncatedSVD

In [7]:
from sklearn.decomposition import TruncatedSVD
import numpy as np

k = 50 
svd = TruncatedSVD(n_components=k)

U = svd.fit_transform(normalized_term_document_matrix)
S = np.diag(svd.singular_values_)  
VT = svd.components_


S_inv = np.linalg.inv(S)

print("Inverse of S:")
print(S_inv)

Inverse of S:
[[0.02287395 0.         0.         ... 0.         0.         0.        ]
 [0.         0.05431071 0.         ... 0.         0.         0.        ]
 [0.         0.         0.05737427 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.16156731 0.         0.        ]
 [0.         0.         0.         ... 0.         0.16313911 0.        ]
 [0.         0.         0.         ... 0.         0.         0.16473821]]


In [8]:
Xq_prime = normalized_term_document_matrix.T

print("Xq_prime shape: ",Xq_prime.shape)
print("U shape: ",U.shape)
print("S_inv shape: ",S_inv.shape)


Xq_prime shape:  (2000, 20782)
U shape:  (20782, 50)
S_inv shape:  (50, 50)


In [9]:
Dq = Xq_prime.dot(U).dot(S_inv)
Dq.shape

(2000, 50)

In [10]:
a=list(df['encoded_topics'].unique())
len(a)

10

In [11]:
class_mapping = {old_label: new_label for new_label, old_label in enumerate(df['encoded_topics'].unique())}
df['encoded_topics'] = df['encoded_topics'].map(class_mapping)
df['encoded_topics'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [12]:
reverse_class_mapping = {v: k for k, v in class_mapping.items()}
print("Old Class Categories:")
for encoded_label, old_label in reverse_class_mapping.items():
    print(f"Encoded Label {encoded_label}: Original Label {old_label}")


Old Class Categories:
Encoded Label 0: Original Label 69
Encoded Label 1: Original Label 4
Encoded Label 2: Original Label 41
Encoded Label 3: Original Label 20
Encoded Label 4: Original Label 55
Encoded Label 5: Original Label 0
Encoded Label 6: Original Label 27
Encoded Label 7: Original Label 6
Encoded Label 8: Original Label 13
Encoded Label 9: Original Label 52


## Model

## Cross Validation

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score

# Initialize variables to store true labels and predicted labels
true_labels = []
predicted_labels = []

num_folds=10
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)# Initialize list to store test accuracies
test_accuracies = []

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(skf.split(Dq, df['encoded_topics'])):
    print(f'Fold {fold + 1}/{num_folds}')

    # Split data into train and test sets for this fold
    X_train, X_test = Dq[train_index], Dq[test_index]
    y_train, y_test = df['encoded_topics'].iloc[train_index], df['encoded_topics'].iloc[test_index]

    num_classes = len(np.unique(np.concatenate((y_train, y_test))))

    y_train_encoded = to_categorical(y_train, num_classes=num_classes)
    y_test_encoded = to_categorical(y_test, num_classes=num_classes)

    X_train_flattened = X_train.reshape((X_train.shape[0], -1))
    X_test_flattened = X_test.reshape((X_test.shape[0], -1))

    # Define the model architecture
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train_flattened.shape[1],)),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Define early stopping criteria
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model with early stopping
    history = model.fit(X_train_flattened, y_train_encoded, epochs=100, batch_size=64, 
                        validation_split=0.2, callbacks=[early_stopping], verbose=0)

    # Evaluate the model on the test set and collect predictions
    test_loss, test_accuracy = model.evaluate(X_test_flattened, y_test_encoded)
    print(f'Test accuracy for fold {fold + 1}: {test_accuracy*100:.2f}%')

    # Store test accuracy for this fold
    test_accuracies.append(test_accuracy)

    # Collect true labels and predicted labels
    true_labels.extend(y_test)
    predicted_probs = model.predict(X_test_flattened)
    predicted_labels.extend(np.argmax(predicted_probs, axis=1))

# Calculate average test accuracy
avg_test_accuracy = np.mean(test_accuracies)

# Generate classification report
report = classification_report(true_labels, predicted_labels)

# Add average accuracy to the report
report_with_avg_accuracy = f'Average Test Accuracy: {avg_test_accuracy*100:.2f}%\n\n{report}'
print("Classification Report with Average Test Accuracy:\n", report_with_avg_accuracy)


Fold 1/10
7/7 [==============================] - 0s 5ms/step - loss: 0.8711 - accuracy: 0.7100
Test accuracy for fold 1: 71.00%
7/7 [==============================] - 0s 5ms/step
Fold 2/10
7/7 [==============================] - 0s 5ms/step - loss: 0.8286 - accuracy: 0.7350
Test accuracy for fold 2: 73.50%
7/7 [==============================] - 0s 5ms/step
Fold 3/10
7/7 [==============================] - 0s 6ms/step - loss: 0.9504 - accuracy: 0.7250
Test accuracy for fold 3: 72.50%
7/7 [==============================] - 0s 6ms/step
Fold 4/10
7/7 [==============================] - 0s 8ms/step - loss: 0.8319 - accuracy: 0.7450
Test accuracy for fold 4: 74.50%
7/7 [==============================] - 0s 6ms/step
Fold 5/10
7/7 [==============================] - 0s 4ms/step - loss: 0.7985 - accuracy: 0.7450
Test accuracy for fold 5: 74.50%
7/7 [==============================] - 0s 6ms/step
Fold 6/10
7/7 [==============================] - 0s 5ms/step - loss: 0.7862 - accuracy: 0.7500
Test accur

In [14]:
from sklearn.metrics import classification_report, accuracy_score

# Initialize variables to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Initialize list to store test accuracies
test_accuracies = []

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(skf.split(Dq, df['encoded_topics'])):
    print(f'Fold {fold + 1}/{num_folds}')

    # Split data into train and test sets for this fold
    X_train, X_test = Dq[train_index], Dq[test_index]
    y_train, y_test = df['encoded_topics'].iloc[train_index], df['encoded_topics'].iloc[test_index]

    num_classes = len(np.unique(np.concatenate((y_train, y_test))))

    y_train_encoded = to_categorical(y_train, num_classes=num_classes)
    y_test_encoded = to_categorical(y_test, num_classes=num_classes)

    X_train_flattened = X_train.reshape((X_train.shape[0], -1))
    X_test_flattened = X_test.reshape((X_test.shape[0], -1))

    # Define the model architecture
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train_flattened.shape[1],)),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Define early stopping criteria
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model with early stopping
    history = model.fit(X_train_flattened, y_train_encoded, epochs=100, batch_size=64, 
                        validation_split=0.2, callbacks=[early_stopping], verbose=0)

    # Evaluate the model on the test set and collect predictions
    test_loss, test_accuracy = model.evaluate(X_test_flattened, y_test_encoded)
    print(f'Test accuracy for fold {fold + 1}: {test_accuracy*100:.2f}%')

    # Store test accuracy for this fold
    test_accuracies.append(test_accuracy)

    # Collect true labels and predicted labels
    true_labels.extend(y_test)
    predicted_probs = model.predict(X_test_flattened)
    predicted_labels.extend(np.argmax(predicted_probs, axis=1))

# Calculate average test accuracy
avg_test_accuracy = np.mean(test_accuracies)

# Generate classification report
report = classification_report(true_labels, predicted_labels)

# Add average accuracy to the report
report_with_avg_accuracy = f'Average Test Accuracy: {avg_test_accuracy*100:.2f}%\n\n{report}'
print("Classification Report with Average Test Accuracy:\n", report_with_avg_accuracy)


Fold 1/10
7/7 [==============================] - 0s 4ms/step - loss: 0.8419 - accuracy: 0.7150
Test accuracy for fold 1: 71.50%
7/7 [==============================] - 0s 6ms/step
Fold 2/10
7/7 [==============================] - 0s 5ms/step - loss: 0.8320 - accuracy: 0.7250
Test accuracy for fold 2: 72.50%
7/7 [==============================] - 0s 6ms/step
Fold 3/10
7/7 [==============================] - 0s 5ms/step - loss: 0.9218 - accuracy: 0.7250
Test accuracy for fold 3: 72.50%
7/7 [==============================] - 0s 6ms/step
Fold 4/10
7/7 [==============================] - 0s 5ms/step - loss: 0.8238 - accuracy: 0.7200
Test accuracy for fold 4: 72.00%
7/7 [==============================] - 0s 4ms/step
Fold 5/10
7/7 [==============================] - 0s 5ms/step - loss: 0.8058 - accuracy: 0.7400
Test accuracy for fold 5: 74.00%
7/7 [==============================] - 0s 5ms/step
Fold 6/10
7/7 [==============================] - 0s 6ms/step - loss: 0.7801 - accuracy: 0.7300
Test accur

## Early stopping

In [15]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping



# Then proceed with train-test split
X_train, X_test, y_train, y_test = train_test_split(
    Dq, 
    df['encoded_topics'], 
    test_size=0.2, 
    random_state=42,
    stratify=df['encoded_topics']  # This ensures stratified splitting
)


num_classes = len(np.unique(y_train))

y_train_encoded = to_categorical(y_train, num_classes=num_classes)
y_test_encoded = to_categorical(y_test, num_classes=num_classes)



X_train_flattened = X_train.reshape((X_train.shape[0], -1))
X_test_flattened = X_test.reshape((X_test.shape[0], -1))

# Define the model architecture
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train_flattened.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train_flattened, y_train_encoded, epochs=100, batch_size=64, 
                    validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_flattened, y_test_encoded)
print(f'Test accuracy: {test_accuracy*100:.2f}%')


Epoch 1/100
20/20 [==============================] - 2s 40ms/step - loss: 2.1895 - accuracy: 0.2648 - val_loss: 2.0112 - val_accuracy: 0.5188
Epoch 2/100
20/20 [==============================] - 0s 13ms/step - loss: 1.8028 - accuracy: 0.5352 - val_loss: 1.5322 - val_accuracy: 0.5938
Epoch 3/100
20/20 [==============================] - 0s 14ms/step - loss: 1.3453 - accuracy: 0.6344 - val_loss: 1.1153 - val_accuracy: 0.6938
Epoch 4/100
20/20 [==============================] - 0s 16ms/step - loss: 1.0708 - accuracy: 0.6727 - val_loss: 0.9630 - val_accuracy: 0.6906
Epoch 5/100
20/20 [==============================] - 0s 13ms/step - loss: 0.9124 - accuracy: 0.7031 - val_loss: 0.8799 - val_accuracy: 0.6938
Epoch 6/100
20/20 [==============================] - 0s 14ms/step - loss: 0.8506 - accuracy: 0.7211 - val_loss: 0.8629 - val_accuracy: 0.7063
Epoch 7/100
20/20 [==============================] - 0s 14ms/step - loss: 0.7894 - accuracy: 0.7422 - val_loss: 0.8265 - val_accuracy: 0.7219
Epoch 

In [16]:
from sklearn.metrics import classification_report
from numpy import argmax, unique
import numpy as np


predictions = model.predict(X_test_flattened)  # Ensure this matches your reshaped test data variable


predictions_int = argmax(predictions, axis=1)


y_test_int = argmax(y_test_encoded, axis=1)


all_classes = unique(np.concatenate((y_test_int, predictions_int)))


target_names = [f'Class {i+1}' for i in all_classes]  


report = classification_report(y_test_int, predictions_int, target_names=target_names)

print(report)


13/13 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

     Class 1       0.84      0.95      0.89        40
     Class 2       0.82      0.68      0.74        40
     Class 3       0.94      0.80      0.86        40
     Class 4       0.78      0.72      0.75        40
     Class 5       0.65      0.65      0.65        40
     Class 6       0.46      0.47      0.47        40
     Class 7       0.69      0.78      0.73        40
     Class 8       0.70      0.65      0.68        40
     Class 9       0.69      0.78      0.73        40
    Class 10       0.88      0.95      0.92        40

    accuracy                           0.74       400
   macro avg       0.75      0.74      0.74       400
weighted avg       0.75      0.74      0.74       400

